# Exploratory Data Analysis


## Dataset
#### [Movies Dataset](https://grouplens.org/datasets/movielens/)
- movies.csv: The main Movies file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include:

    - budget - The budget in which the movie was made.
    - genre - The genre of the movie, Action, Comedy ,Thriller etc.
    - homepage - A link to the homepage of the movie.
    - id - This is infact the movie_id as in the first dataset.
    - keywords - The keywords or tags related to the movie.
    - original_language - The language in which the movie was made.
    - original_title - The title of the movie before translation or adaptation.
    - overview - A brief description of the movie.
    - popularity - A numeric quantity specifying the movie popularity.
    - production_companies - The production house of the movie.
    - production_countries - The country in which it was produced.
    - revenue - The worldwide revenue generated by the movie.
    - release_date - The date on which it was released.
    - runtime - The running time of the movie in minutes.
    - status - "Released" or "Rumored".
    - tagline - Movie's tagline.
    - title - Title of the movie.
    - vote_average - average ratings the movie recieved.
    - vote_count - the count of votes recieved.
    
    
- credits.csv: 
   - movieId - unique identifier for each movie
   - title - name of movie
   - crew - behind the scences crew
   - cast - actors



In [236]:
  
import pandas as pd
import numpy as np




## Import Data

In [237]:

movies = pd.read_csv('..//data//raw//tmdb_5000_movies.csv')
credit = pd.read_csv('..//data//raw//tmdb_5000_credits.csv')



In [238]:
movies = movies.merge(credit, on = 'title')

#### Movie Dataset Analysis

In [239]:
movies.head(2)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.44,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.00,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.20,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.08,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.00,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.90,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [240]:
initial_movie_shape = movies.shape
initial_movie_shape

(4809, 23)

#### Missing values

In [241]:
round(movies.isnull().sum()/len(movies) * 100),2

(budget                  0.00
 genres                  0.00
 homepage               64.00
 id                      0.00
 keywords                0.00
 original_language       0.00
 original_title          0.00
 overview                0.00
 popularity              0.00
 production_companies    0.00
 production_countries    0.00
 release_date            0.00
 revenue                 0.00
 runtime                 0.00
 spoken_languages        0.00
 status                  0.00
 tagline                18.00
 title                   0.00
 vote_average            0.00
 vote_count              0.00
 movie_id                0.00
 cast                    0.00
 crew                    0.00
 dtype: float64,
 2)

#### 

#### Dropped columns

We have a lot of unnecessary columns that are unnecessary, redundant or are missing values. 

In [242]:
movies.drop(['homepage','tagline', 'spoken_languages', 'status', 'production_countries', 'production_companies', 'original_title', 'original_language', 'budget', 'id', 'release_date','revenue', 'overview', 'runtime'], axis=1, inplace=True)


In [243]:
movies.columns

Index(['genres', 'keywords', 'popularity', 'runtime', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

#### Genre Analysis

The genres in the genres column are treated as a dictionary with string literals. We need to create tags.

In [244]:
import ast


def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L


In [245]:
movies['genres'] = movies['genres'].apply(convert)


#### Check for Movies with missing genres

In [246]:
movies.drop(movies[movies['genres'].str.len() == 0].index, inplace=True)

In [247]:
all_genres = sum(movies.genres, [])
print(f'We have {len(set(all_genres))} unique genres in our dataset.')


We have 20 unique genres in our dataset.


In [248]:
import nltk
all_genres = nltk.FreqDist(all_genres)
all_genres_df = pd.DataFrame(
    {'Genre': list(all_genres.keys()), 'Count': list(all_genres.values())})


#### Genres Bar Graph

In [249]:
import plotly.express as px

fig = px.bar(all_genres_df, x='Genre', y='Count', title='# of Genres' )
fig.show()

It looks like the top 3 genres in our movie dataset are Drama, Comedy and Thriller. 

#### Keywords

It appears that our Keywords columns suffers from the same issue that our genre column was facing.

In [250]:
movies['keywords'] = movies['keywords'].apply(convert)


#### Vote Average

In [ ]:
outOfRangeVotes = movies[(movies.vote_average < 0) |
                         (movies.vote_average > 10)]
outOfRangeVotes


The vote_average column should contain values that range from 0-10. It we don't have any votes out of range.

#### Cast

Each movie can contains hundreds of cast so we'll limit the cast to 10.

In [255]:
def convertCast(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 10:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L


In [256]:
movies['cast'] = movies['cast'].apply(convertCast)

#### Director

In [259]:
def get_Director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L


In [260]:
movies['crew'] = movies['crew'].apply(get_Director)


In [261]:
movies['crew']

0           [James Cameron]
1          [Gore Verbinski]
2              [Sam Mendes]
3       [Christopher Nolan]
4          [Andrew Stanton]
               ...         
4803     [Neill Dela Llana]
4804     [Robert Rodriguez]
4805         [Edward Burns]
4806          [Scott Smith]
4808     [Brian Herzlinger]
Name: crew, Length: 4781, dtype: object

#### Top 10 Highest Rated Movies

In [262]:
movies.groupby('title')['vote_average'].mean().sort_values(ascending=False).head(10)

title
Little Big Top             10.00
Stiff Upper Lips           10.00
Me You and Five Bucks      10.00
Dancer, Texas Pop. 81      10.00
One Man's Hero              9.30
There Goes My Baby          8.50
The Shawshank Redemption    8.50
The Godfather               8.40
The Prisoner of Zenda       8.40
Counting                    8.30
Name: vote_average, dtype: float64

#### Movies with the most Ratings

In [263]:
movies.groupby('title')['vote_count'].count().sort_values(ascending=False).head(10)

title
Out of the Blue                          4
The Host                                 4
Batman                                   4
#Horror                                  1
Spun                                     1
Spy Kids 3-D: Game Over                  1
Spy Kids 2: The Island of Lost Dreams    1
Spy Kids                                 1
Spy Hard                                 1
Spy Game                                 1
Name: vote_count, dtype: int64

In [266]:
print(f'Our initial movie and credit dataset had {initial_movie_shape} rows and columns')
print(f'Our final movie and credit dataset has {movies.shape} rows and columns')

Our initial movie and credit dataset had (4809, 23) rows and columns
Our final movie and credit dataset has (4781, 10) rows and columns
